In [2]:
import pandas as pd
import numpy as np
from IPython.display import display

house_prices = pd.read_csv("data/house-prices.csv")

## Get all columns except last

In [3]:
house_prices.iloc[:, :-1].head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


## Get last column

In [4]:
house_prices.iloc[:, -1:].head()

,SalePrice
0,208500
1,181500
2,223500
3,140000
4,250000


## Converting a data frame into an array

In [5]:
df = pd.DataFrame({"price": [10, 20, 30]})
display(df)

print("Method 1 using values:\n", df.values)
print("\nMethod 2 using np.array:\n", np.array(df))
print("\nCasting to a float with np.array:\n", np.array(df, np.float32))

,price
0,10
1,20
2,30


Method 1 using values:
 [[10]
 [20]
 [30]]

Method 2 using np.array:
 [[10]
 [20]
 [30]]

Casting to a float with np.array:
 [[10.]
 [20.]
 [30.]]


## Iterating over rows

In [6]:
df = pd.DataFrame({"price": [10, 20, 30]})

for index, row in df.iterrows():
    print(row["price"])

10
20
30


## Removing NA Values

In [7]:
df = pd.DataFrame({
    "first": ["Matt", "Eli", "Tom"],
    "initial": ["H", np.nan, "S"],
    "last": ["Mazur", "Smith", np.nan]
})

print("Original:\n\n", df)
print("\nRemoving all rows with NA:\n\n", df.dropna())
print("\nRemoving just last names with NA:\n\n", df.dropna(subset=["last"]))
print("\nRemoving all columns with NA:\n\n", df.dropna(axis=1))

Original:

   first initial   last
0  Matt       H  Mazur
1   Eli     NaN  Smith
2   Tom       S    NaN

Removing all rows with NA:

   first initial   last
0  Matt       H  Mazur

Removing just last names with NA:

   first initial   last
0  Matt       H  Mazur
1   Eli     NaN  Smith

Removing all columns with NA:

   first
0  Matt
1   Eli
2   Tom


### Counting missing values

In [8]:
df.isna().sum()

first      0
initial    1
last       1
dtype: int64

### Measuring percent of values in each column that are missing

In [9]:
percent_missing = df.isna().sum() / len(df)
percent_missing

first      0.000000
initial    0.333333
last       0.333333
dtype: float64

### Filtering columns with a high percent of missing values

In [10]:
mask = percent_missing < 0.3
df_filtered = df.loc[:, mask]
print("\nAfter removing columns with 30%+ missing values:")
display(df_filtered)


After removing columns with 30%+ missing values:


,first
0,Matt
1,Eli
2,Tom


## Lagged Values

In [11]:
df = pd.DataFrame({
    "price": range(1, 11)
})
df["last_price"] = df["price"].shift()
df

,price,last_price
0,1,NaN
1,2,1.0
2,3,2.0
3,4,3.0
4,5,4.0
5,6,5.0
6,7,6.0
7,8,7.0
8,9,8.0
9,10,9.0


## Lagged Values in a Group with Trailing Averages

In [12]:
df = pd.DataFrame({
    "stock": ["AAPL", "AAPL", "AAPL", "GOOG", "GOOG", "GOOG", "GOOG", "T", "T", "T"],
    "price": range(1, 11)
})

df["last_price"] = df.groupby("stock")["price"].shift()

# This row and the row prior to it
df["mean_price_including_current"] = df.groupby("stock")["price"].rolling(2).mean().reset_index(0, drop=True)

# The two prior rows (and not this one)
df["mean_price_not_including_current"] = df.groupby("stock")["last_price"].rolling(2).mean().reset_index(0, drop=True)

# If there are two prior values, it will average them, but 1 is fine
df["mean_price_not_including_current_no_min"] = df.groupby("stock")["last_price"].rolling(2, min_periods=1).mean().reset_index(0, drop=True)

df

,stock,price,last_price,mean_price_including_current,mean_price_not_including_current,mean_price_not_including_current_no_min
0,AAPL,1,NaN,NaN,NaN,NaN
1,AAPL,2,1.0,1.5,NaN,1.0
2,AAPL,3,2.0,2.5,1.5,1.5
3,GOOG,4,NaN,NaN,NaN,NaN
4,GOOG,5,4.0,4.5,NaN,4.0
5,GOOG,6,5.0,5.5,4.5,4.5
6,GOOG,7,6.0,6.5,5.5,5.5
7,T,8,NaN,NaN,NaN,NaN
8,T,9,8.0,8.5,NaN,8.0
9,T,10,9.0,9.5,8.5,8.5


## Subsetting

In [13]:
df = pd.DataFrame({
    "index": range(1, 11),
    "price": np.random.randint(0, 100, size=10),
    "other": np.random.randint(1000, 2000, size=10)
}).set_index("index")

display(df)

select_rows_by_column_value = df[df["price"] > 50]
display(select_rows_by_column_value)

select_rows_by_index_value = df.loc[[3, 4, 5, 6, 7]]
display(select_rows_by_index_value)

select_rows_by_index_value = df[df.index.isin([3, 4, 5, 6, 7])]
display(select_rows_by_index_value)

select_rows_and_columns = df.loc[df["price"] > 50, ["price"]]
display(select_rows_and_columns)


,price,other
index,,
1,75,1070
2,19,1517
3,97,1860
4,77,1598
5,10,1652
6,49,1298
7,53,1000
8,11,1420
9,83,1877


,price,other
index,,
1,75,1070
3,97,1860
4,77,1598
7,53,1000
9,83,1877


,price,other
index,,
3,97,1860
4,77,1598
5,10,1652
6,49,1298
7,53,1000


,price,other
index,,
3,97,1860
4,77,1598
5,10,1652
6,49,1298
7,53,1000


,price
index,
1,75
3,97
4,77
7,53
9,83


In [14]:
df = pd.DataFrame({
    "feature1": [1, 2, 3],
    "feature2": [4, 5, 6],
    "target": [7, 8, 9]
})

print("Data Frame:")
display(df)

X = df.iloc[:,:-1]
y = df.iloc[:,-1]    # Series
# y = df.iloc[:,[-1]]  # Data Frame

print("Features:")
display(X)

print("\nTarget:")
display(y)

Data Frame:


,feature1,feature2,target
0,1,4,7
1,2,5,8
2,3,6,9


Features:


,feature1,feature2
0,1,4
1,2,5
2,3,6



Target:


0    7
1    8
2    9
Name: target, dtype: int64

## Updating values of a subset

In [15]:
df = pd.DataFrame({
    "stock": ["AAPL", "GOOG"],
    "price": [5, 40]
})

display(df)

# Double the price of values that meet a certain criteria
df.loc[df["stock"] == "GOOG", "price"] = df.loc[df["stock"] == "GOOG", "price"] * 2

display(df)

,stock,price
0,AAPL,5
1,GOOG,40


,stock,price
0,AAPL,5
1,GOOG,80


## Filling NA values

In [16]:
df = pd.DataFrame({
    "index": range(1, 6),
    "price": [20, 40, np.nan, 60, np.nan]
}).set_index("index")

display(df)

# Avoid using inplace
df.loc[3] = df.loc[3].fillna(0)

display(df)

,price
index,
1,20.0
2,40.0
3,NaN
4,60.0
5,NaN


,price
index,
1,20.0
2,40.0
3,0.0
4,60.0
5,NaN


## Summing Columns Containing NAs

In [17]:
df = pd.DataFrame({
    "a": [np.nan, 4, 5],
    "b": [10, np.nan, 6]
})

display(df)

df["sum"] = df["a"].fillna(0) + df["b"].fillna(0)

display(df)

,a,b
0,NaN,10.0
1,4.0,NaN
2,5.0,6.0


,a,b,sum
0,NaN,10.0,10.0
1,4.0,NaN,4.0
2,5.0,6.0,11.0


## Joining with shared indeces

In [18]:
df1 = pd.DataFrame({
    "company": ["AAPL", "GOOG"],
    "price": [10, 20]
}).set_index("company")

df2 = pd.DataFrame({
    "company": ["AAPL", "GOOG"],
    "ceo": ["Tim Cook", "Sundar Pichai"]
}).set_index("company")

display(df1)
display(df2)

# Using join
df3 = df1.join(df2)
display(df3)

# Using concat
df4 = pd.concat([df1, df2], axis=1)
display(df4)

,price
company,
AAPL,10
GOOG,20


,ceo
company,
AAPL,Tim Cook
GOOG,Sundar Pichai


,price,ceo
company,,
AAPL,10,Tim Cook
GOOG,20,Sundar Pichai


,price,ceo
company,,
AAPL,10,Tim Cook
GOOG,20,Sundar Pichai


## Joining without shared indeces

In [19]:
df1 = pd.DataFrame({
    "company": ["AAPL", "GOOG", "TSLA"],
    "price": [10, 20, 30]
})

df2 = pd.DataFrame({
    "company": ["AAPL", "GOOG"],
    "ceo": ["Tim Cook", "Sundar Pichai"]
})

# Use `merge` instead of `join`
df3 = pd.merge(df1, df2, on="company")
display(df3)

,company,price,ceo
0,AAPL,10,Tim Cook
1,GOOG,20,Sundar Pichai


## Viewing column names

In [20]:
df = pd.DataFrame({
    "company": ["AAPL", "GOOG", "TSLA"],
    "price": [10, 20, 30]
})

# to_list is an alias of tolist
print("Columns:", df.columns.tolist())
print("Columns:", df.columns.to_list())

Columns: ['company', 'price']
Columns: ['company', 'price']


## Grouping

In [21]:
df = pd.DataFrame({
    "stock": ["AAPL", "AAPL", "AAPL", "GOOG", "GOOG", "GOOG", "GOOG", "T", "T", "T"],
    "price": range(1, 11)
})

display(df)

with_index = df.groupby("stock")["price"].mean()
print("With Index:\n\n", with_index)

without_index = df.groupby("stock", as_index=False)["price"].mean()
print("\nWithout Index:\n\n", without_index)


,stock,price
0,AAPL,1
1,AAPL,2
2,AAPL,3
3,GOOG,4
4,GOOG,5
5,GOOG,6
6,GOOG,7
7,T,8
8,T,9
9,T,10


With Index:

 stock
AAPL    2.0
GOOG    5.5
T       9.0
Name: price, dtype: float64

Without Index:

   stock  price
0  AAPL    2.0
1  GOOG    5.5
2     T    9.0


## Setting uncommon categorical values to Other

Source: [Handling Rare Categorical Values in Pandas](https://medium.com/gett-engineering/handling-rare-categorical-values-in-pandas-d1e3f17475f0)

In [22]:
df = pd.DataFrame({"solution": ["email", "email", "email", "email", "email", "chat", "chat", "chat", "chat", "phone", "competitor"]})
display(df)

,solution
0,email
1,email
2,email
3,email
4,email
5,chat
6,chat
7,chat
8,chat
9,phone


In [23]:
# We can see the value counts by % of total
value_percents = df["solution"].value_counts(normalize=True)
value_percents

email         0.454545
chat          0.363636
competitor    0.090909
phone         0.090909
Name: solution, dtype: float64

In [24]:
# Then we can map the values to these percents
df["solution"].map(value_percents)

0     0.454545
1     0.454545
2     0.454545
3     0.454545
4     0.454545
5     0.363636
6     0.363636
7     0.363636
8     0.363636
9     0.090909
10    0.090909
Name: solution, dtype: float64

In [25]:
# And create a mask representing which ones are less than some threshold
mask = df["solution"].map(value_percents) < 0.25
mask

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9      True
10     True
Name: solution, dtype: bool

In [26]:
# And then use the mask method and specify the `other` argument
df["solution"].mask(mask, 'Other')

0     email
1     email
2     email
3     email
4     email
5      chat
6      chat
7      chat
8      chat
9     Other
10    Other
Name: solution, dtype: object

In [27]:
# Putting it all togther:

df["solution_categorized"] = df["solution"].mask(df["solution"].map(df["solution"].value_counts(normalize=True)) < 0.25, "Other")
df

,solution,solution_categorized
0,email,email
1,email,email
2,email,email
3,email,email
4,email,email
5,chat,chat
6,chat,chat
7,chat,chat
8,chat,chat
9,phone,Other


## Selecting or excluding columns with specific types

In [28]:
df = pd.DataFrame({
    "name": ["Matt", "Crystal"],
    "age": [35, 35]
})

display(df)

print("Including strings:")
display(df.select_dtypes(include="object"))

print("Excluding strings:")
display(df.select_dtypes(exclude="object"))

,name,age
0,Matt,35
1,Crystal,35


Including strings:


,name
0,Matt
1,Crystal


Excluding strings:


,age
0,35
1,35


## Getting indeces of categorical columns

### Method 1:

In [29]:
np.where(house_prices.dtypes == np.object)

(array([ 2,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 21, 22, 23, 24,
        25, 27, 28, 29, 30, 31, 32, 33, 35, 39, 40, 41, 42, 53, 55, 57, 58,
        60, 63, 64, 65, 72, 73, 74, 78, 79]),)

In [30]:
np.where(house_prices.dtypes == np.object)[0]

array([ 2,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 21, 22, 23, 24,
       25, 27, 28, 29, 30, 31, 32, 33, 35, 39, 40, 41, 42, 53, 55, 57, 58,
       60, 63, 64, 65, 72, 73, 74, 78, 79])

### Method 2:

In [31]:
categorical_features = house_prices.select_dtypes(include="object").columns
print("Categorical Features:\n\n", categorical_features)

categorical_feature_indeces = [house_prices.columns.get_loc(column) for column in categorical_features]
print("\nCategorical Features Indeces:\n\n", categorical_feature_indeces)

Categorical Features:

 Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')

Categorical Features Indeces:

 [2, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 35, 39, 40, 41, 42, 53, 55, 57, 58, 60, 63, 64, 65, 72, 73, 74, 78, 79]


## Filtering to n largest column values for a specific column

In [32]:
# Note that the results are sorted by descending sale price
house_prices.nlargest(10, "SalePrice")

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
691,692,60,RL,104.0,21535,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,1,2007,WD,Normal,755000
1182,1183,60,RL,160.0,15623,Pave,NaN,IR1,Lvl,AllPub,...,555,Ex,MnPrv,NaN,0,7,2007,WD,Abnorml,745000
1169,1170,60,RL,118.0,35760,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,7,2006,WD,Normal,625000
898,899,20,RL,100.0,12919,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,3,2010,New,Partial,611657
803,804,60,RL,107.0,13891,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,1,2009,New,Partial,582933
1046,1047,60,RL,85.0,16056,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,7,2006,New,Partial,556581
440,441,20,RL,105.0,15431,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2009,WD,Normal,555000
769,770,60,RL,47.0,53504,Pave,NaN,IR2,HLS,AllPub,...,0,NaN,NaN,NaN,0,6,2010,WD,Normal,538000
178,179,20,RL,63.0,17423,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,7,2009,New,Partial,501837
798,799,60,RL,104.0,13518,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,7,2009,New,Partial,485000


In [35]:
df = pd.DataFrame({
    "stock": ["A", "B", "A"]
})

df["value"] = np.where(df["stock"] == "A", 4, 5)
df

,stock,value
0,A,4
1,B,5
2,A,4
